In [1]:
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from openpyxl.styles import Alignment

# -----------------------------
# Hjælpefunktioner
# -----------------------------

def extract_text_recursive(elem):
    """Udtrækker sammenhængende tekst fra et XML-element (inkl. <hi> mv.)."""
    return ' '.join(elem.itertext()).strip()

def find_prioritised_year(text):
    """Finder årstal (1798–1872) nært bestemte nøgleord (affattelsestid/datering)."""
    if not text:
        return None, False
    text_lower = text.lower()
    context_keywords = ["affattelsestid", "datering"]
    for kw in context_keywords:
        if kw in text_lower:
            # Find ord omkring keyword (indenfor ±20 ord)
            words = text.split()
            for i, word in enumerate(words):
                if kw in word.lower():
                    context = words[max(0, i-10):i+20]
                    matches = re.findall(r'(1[78][0-9]{2})', ' '.join(context))
                    for y in matches:
                        y_int = int(y)
                        if 1798 <= y_int <= 1872:
                            return y_int, True
    return None, False

def fallback_year(text):
    """Første gyldige årstal (1798–1872) i hele teksten, ellers 'Ukendt'."""
    matches = re.findall(r'(1[78][0-9]{2})', text or "")
    for y in matches:
        y_int = int(y)
        if 1798 <= y_int <= 1872:
            return y_int
    return "Ukendt"

def extract_dating(xml_root):
    """Går igennem prioriterede felter og forsøger at finde et årstal."""
    priority_tags = [
        "Indholdsregest",
        "title_el_kort_indholdsangivelse",
        "Tidl_forskeres_skoensmaessige_detering_er",
        "Ms_forste_linie",
        "Ms_sidste_linie",
        "Tryk_el_benyttelse",
    ]
    for tag in priority_tags:
        for el in xml_root.findall(f".//{tag}"):
            text = extract_text_recursive(el)
            year, prioritized = find_prioritised_year(text)
            if year:
                return year, tag, text
            year = fallback_year(text)
            if year != "Ukendt":
                return year, tag, text
    return "Ukendt", "", ""

def extract_number(root, tag):
    """
    Udtrækker tal fra et simpelt XML-tag (fx Antal_sider/Antal_blade/Nr_i_bibliografien).
    Regler:
      1) Hvis feltet starter med et interval (1-10 / 1–10 / 1—10), returneres (slut - start + 1).
      2) Ellers returneres første heltal i starten, afgrænset af mellemrum eller .,;:
    """
    el = root.find(f".//{tag}")
    if el is not None:
        p = el.find("p")
        text = p.text if p is not None and p.text else el.text
        if text:
            s = text.strip()
            # Normaliser bindestreger (en-dash/em-dash) til '-'
            s_norm = s.replace("–", "-").replace("—", "-")
            # 1) Interval i starten, fx '1-10' efterfulgt af evt. separator/strengslut
            m_range = re.match(r'^(\d+)\s*-\s*(\d+)(?=[\s\.,;:]|$)', s_norm)
            if m_range:
                a, b = int(m_range.group(1)), int(m_range.group(2))
                if b >= a:
                    return str(b - a + 1)
            # 2) Enkelt heltal i starten
            m_single = re.match(r'^(\d+)(?=[\s\.,;:]|$)', s)
            if m_single:
                return m_single.group(1)
    return ""


Behandler filer: 100%|███████████████████████████████████████████████████████████| 4872/4872 [00:04<00:00, 1056.89it/s]


✅ Data gemt: C:\Users\au468045\OneDrive - Aarhus universitet\Desktop\PhD\Artikler\Variants-ESTS-2025\python-test\grundtvig_datering_alle_biblio.xlsx
ℹ️ Antal rækker hvor antal_sider blev beregnet fra antal_blade: 199
